## Segmenting and Clustering Toronto's Neighborhoods

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans



print('Libraries imported.')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: | 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                                       /                                  failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - cffi -> python[version='2.7.*|3.5.*|3.6.*|3.6.9|3.6.9|3.6.9|3.6.9|>=3.6,<3.7.0a0|>=3.9,<3.10.0a0|>=3.8,<3.9.0a0|>=3.7,<3.8.0a0|>=2.7,<2.8.0a0|>=3.5,<3.6.0a0|3.4.*',build='1_73_pypy|3_73_pypy|2_73_pypy|0_73_pypy']
  - geopy -> python[version='2.7.*|3.4.*|3.5.*|3.6.*']
  - google-auth-oauthlib -> pytho

Package s3transfer conflicts for:
ibm-wsrt-py37main-main -> s3transfer==0.3.3[build=*]
boto3 -> s3transfer[version='>=0.1.0,<0.2.0|>=0.1.10,<0.2.0|>=0.2.0,<0.3.0|>=0.3.0,<0.4.0']

Package mpmath conflicts for:
ibm-wsrt-py37main-main -> mpmath==1.1.0[build=*]
sympy -> mpmath[version='>=0.19']

Package pexpect conflicts for:
ibm-wsrt-py37main-main -> pexpect==4.8.0[build=*]
ipython -> pexpect[version='>4.3']
Note that strict channel priority may have removed packages required for satisfiability.

Libraries imported.


### Load Data from the webpage

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df_list = pd.read_html(url)
df_list

[    Postal Code           Borough  \
 0           M1A      Not assigned   
 1           M2A      Not assigned   
 2           M3A        North York   
 3           M4A        North York   
 4           M5A  Downtown Toronto   
 5           M6A        North York   
 6           M7A  Downtown Toronto   
 7           M8A      Not assigned   
 8           M9A         Etobicoke   
 9           M1B       Scarborough   
 10          M2B      Not assigned   
 11          M3B        North York   
 12          M4B         East York   
 13          M5B  Downtown Toronto   
 14          M6B        North York   
 15          M7B      Not assigned   
 16          M8B      Not assigned   
 17          M9B         Etobicoke   
 18          M1C       Scarborough   
 19          M2C      Not assigned   
 20          M3C        North York   
 21          M4C         East York   
 22          M5C  Downtown Toronto   
 23          M6C              York   
 24          M7C      Not assigned   
 25         

#### Get the desired table

In [20]:
df_table = df_list[0]
df_table

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


#### Remove the rows where boroughs are 'Not assigned'

In [21]:
df_table.drop(df_table[df_table['Borough'] == 'Not assigned'].index, inplace= True) 
df_table

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


#### Reset index of the dataframe

In [22]:
df1= df_table.reset_index(drop=True)
df1

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


#### Number of rows and columns in the dataframe

In [23]:
df1.shape

(103, 3)

In [15]:
pip install geocoder

     |████████████████████████████████| 98 kB 8.1 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [17]:
import geocoder
from geopy.geocoders import Nominatim 

Latitude = np.zeros(df1.shape[0])
Longitude = np.zeros(df1.shape[0])

In [18]:
for i in range (df1.shape[0]):
    Lat = None
    Long = None
    
    while (Lat is None) and (Long is None):
        address = str(df1['Postal Code'][i] + df1['Borough'][i] + 'canada')
        
        geolocator = Nominatim(user_agent = "to_explorer")
        location = geolocator.geocode(address)
        
        Lat = location.Latitude
        Long = location.Longitude
        
        Latitude[i] = Lat
        Longitude[i] = Long
                                                  
                                                  

AttributeError: 'NoneType' object has no attribute 'Latitude'

In [ ]:
def get_geocoder(postal_code_from_df):
     # initialize your variable to None
     lat_lng_coords = None
     # loop until you get the coordinates
     while(lat_lng_coords is None):
       g = geocoder.google('{}, Toronto, Ontario'.format('Postal Code'))
       lat_lng_coords = g.latlng
     latitude = lat_lng_coords[0]
     longitude = lat_lng_coords[1]
     return latitude,longitude



for i in range(0,len(df1)):
    df1['Latitude'][i],df1['Longitude'][i]=get_geocoder(df1.iloc[i]['Postal Code'])

### Get the coordinates of the Toronto's postal codes using the csv file

In [24]:
path = "http://cocl.us/Geospatial_data"
df_cordinates = pd.read_csv(path)
df_cordinates

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


### Merge the Postal Code Coordinates data with Toronto's Borough and Neighborhood data

In [25]:
toronto = pd.merge(df1, df_cordinates, how ='left', on = 'Postal Code')
toronto

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


### Using Geopy library to get coordinates of Toronto City

In [26]:
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.65238435, -79.38356765.


In [34]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library

     |████████████████████████████████| 93 kB 4.0 MB/s  eta 0:00:01


### Toronto Map with superimposed boroughs

In [36]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### To simplify, let's map the borough "Scarborough" and explore neighborhoods in Scarborough

In [37]:
scarborough_data = toronto[toronto['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Get coordinates of Scarborough

In [38]:
address = 'Scarborough, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


### Map of Scarborough with superimposed neighborhoods on top of it.

In [39]:
# create map of Manhattan using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

### Specify Foursquare Credentials

In [40]:
CLIENT_ID = '5H4EEPLPOSTAG41MYOR4JGKJ1DBRHSIKTKAJP0EH5RHQ3CTU' # your Foursquare ID
CLIENT_SECRET = '2OVKFUM3FA302LVTO4ANLNPBTJ0QA3AJH030EJVT4WSOPT3H' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5H4EEPLPOSTAG41MYOR4JGKJ1DBRHSIKTKAJP0EH5RHQ3CTU
CLIENT_SECRET:2OVKFUM3FA302LVTO4ANLNPBTJ0QA3AJH030EJVT4WSOPT3H


### Let's explore Neighborhoods in Scarborough
#### Create a function to get nearby venues in all neighborhoods 

In [96]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Run the above function on each neighborhood of the scarborough and create a new dataframe

In [97]:
scarborough_venues = getNearbyVenues(names= scarborough_data['Neighbourhood'], latitudes=scarborough_data['Latitude'], longitudes=scarborough_data['Longitude'], radius =500)

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge


#### Let's check the size of the dataframe

In [98]:
print(scarborough_venues.shape)
scarborough_venues.head()

(89, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


#### How many venues were returned were each neighborhood?

In [100]:
scarborough_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,5,5,5,5,5,5
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clarks Corners, Tam O'Shanter, Sullivan",12,12,12,12,12,12
"Cliffside, Cliffcrest, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Wexford Heights, Scarborough Town Centre",5,5,5,5,5,5
"Golden Mile, Clairlea, Oakridge",10,10,10,10,10,10
"Guildwood, Morningside, West Hill",8,8,8,8,8,8
"Kennedy Park, Ionview, East Birchmount Park",4,4,4,4,4,4


#### How many unique values are curated from the returned venues?

In [101]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 55 uniques categories.


### Analyze each Neighborhood

In [102]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighbourhood'] = scarborough_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighbourhood,Accessories Store,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Construction & Landscaping,Department Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Gym,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Jewelry Store,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Train Station,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Examine the size of new dataframe

In [103]:
scarborough_onehot.shape

(89, 56)

#### Let's group each neighborhood by using mean frequency of the occurence of each category

In [104]:
scarborough_grouped = scarborough_onehot.groupby('Neighbourhood').mean().reset_index()
scarborough_grouped

,Neighbourhood,Accessories Store,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Construction & Landscaping,Department Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Gym,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Jewelry Store,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Train Station,Vietnamese Restaurant
0,Agincourt,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.200000,0.0,0.0,0.00,0.000,0.000000,0.2,0.000000,0.00,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.2,0.2,0.000,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.20,0.000000,0.0,0.000000,0.00,0.0
1,"Birch Cliff, Cliffside West",0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.25,0.000,0.000000,0.0,0.000000,0.25,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.25,0.000000,0.0,0.000000,0.00,0.0
2,Cedarbrae,0.000000,0.0,0.125,0.000000,0.125000,0.125000,0.0,0.000000,0.0,0.0,0.00,0.125,0.000000,0.0,0.000000,0.00,0.0,0.00,0.000,0.000000,0.125000,0.125000,0.00,0.000000,0.000000,0.125,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.000000,0.0,0.125000,0.00,0.0
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.000000,0.0,0.000,0.000000,0.000000,0.083333,0.0,0.000000,0.0,0.0,0.00,0.000,0.083333,0.0,0.000000,0.00,0.0,0.00,0.000,0.083333,0.083333,0.083333,0.00,0.000000,0.000000,0.000,0.00,0.0,0.000000,0.083333,0.083333,0.000000,0.00,0.0,0.0,0.000,0.0,0.000,0.000000,0.0,0.083333,0.000000,0.0,0.083333,0.166667,0.000000,0.000,0.000,0.000000,0.00,0.000000,0.0,0.083333,0.00,0.0
4,"Cliffside, Cliffcrest, Scarborough Village West",0.000000,0.5,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000,0.000000,0.0,0.000000,0.00,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000,0.0,0.000,0.000000,0.5,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.0
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000,0.200000,0.0,0.000000,0.00,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.00,0.0,0.400000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.2
6,"Golden Mile, Clairlea, Oakridge",0.000000,0.0,0.000,0.000000,0.200000,0.000000,0.0,0.000000,0.2,0.1,0.00,0.000,0.000000,0.0,0.000000,0.00,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.00,0.1,0.000000,0.100000,0.000000,0.000000,0.00,0.0,0.0,0.000,0.1,0.000,0.000000,0.0,0.000000,0.100000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.000000,0.1,0.000000,0.00,0.0
7,"Guildwood, Morningside, West Hill",0.000000,0.0,0.000,0.000000,0.000000,0.125000,0.0,0.125000,0.0,0.0,0.00,0.000,0.000000,0.0,0.000000,0.00,0.0,0.00,0.125,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.00,0.0,0.000000,0.125000,0.000000,0.000000,0.00,0.0,0.0,0.125,0.0,0.125,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.125,0.125,0

#### Check the size of new dataframe

In [105]:
scarborough_grouped.shape

(16, 56)

### Print each neighborhood with top 5 most common venues

In [106]:
num_top_venues = 5

for hood in scarborough_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant   0.2
1                     Lounge   0.2
2               Skating Rink   0.2
3             Breakfast Spot   0.2
4             Clothing Store   0.2


----Birch Cliff, Cliffside West----
                   venue  freq
0           Skating Rink  0.25
1  General Entertainment  0.25
2                   Café  0.25
3        College Stadium  0.25
4      Accessories Store  0.00


----Cedarbrae----
                 venue  freq
0   Athletics & Sports  0.12
1  Fried Chicken Joint  0.12
2               Bakery  0.12
3                 Bank  0.12
4      Thai Restaurant  0.12


----Clarks Corners, Tam O'Shanter, Sullivan----
                  venue  freq
0           Pizza Place  0.17
1    Italian Restaurant  0.08
2  Fast Food Restaurant  0.08
3   Fried Chicken Joint  0.08
4           Gas Station  0.08


----Cliffside, Cliffcrest, Scarborough Village West----
                 venue  freq
0  American Restaurant   0.5
1       

### Let's put it in a dataframe

#### Create a function to sort venues in descending order

In [107]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Creat new dataframe and display top 10 most common venues

In [108]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = scarborough_grouped['Neighbourhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Clothing Store,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Gas Station
1,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Gym,Grocery Store,Gas Station,Fried Chicken Joint
2,Cedarbrae,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,College Stadium,Gym
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Chinese Restaurant,Noodle House,Thai Restaurant,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Intersection,Bank,Italian Restaurant
4,"Cliffside, Cliffcrest, Scarborough Village West",Motel,American Restaurant,Vietnamese Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


### Cluster Neighborhoods

#### Run k-Means to cluster the neighborhoods in 5 clusters

In [109]:
# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 0, 1, 1, 1, 3, 4], dtype=int32)

#### New dataframe including ll clusters and top 10 venues in each neighborhood

In [110]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarborough_merged = scarborough_data

# merge scarborough_grouped with scarborough_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

scarborough_merged.head() 

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4.0,Fast Food Restaurant,Vietnamese Restaurant,Clothing Store,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Department Store
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,2.0,Bar,Construction & Landscaping,Vietnamese Restaurant,Clothing Store,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1.0,Rental Car Location,Electronics Store,Medical Center,Intersection,Bank,Restaurant,Mexican Restaurant,Breakfast Spot,College Stadium,Construction & Landscaping
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3.0,Coffee Shop,Korean Restaurant,Mexican Restaurant,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,College Stadium,Gym


#### Exammining any NaN values in data set before mapping clusters on Scarborough map

In [147]:
#scarborough_merged.dtypes

scarborough_merged.isnull()


,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


#### Removing NaN values from the dataframe

In [149]:
scarborough_merged.dropna(subset=['Cluster Labels'], axis=0, inplace = True)
scarborough_merged.isnull()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


#### Cluster Lables column needs to be in "int" data type to be accepted by the folium map marker

In [151]:
scarborough_merged['Cluster Labels'] = scarborough_merged['Cluster Labels'].astype(int)
scarborough_merged.dtypes

Postal Code                object
Borough                    object
Neighbourhood              object
Latitude                  float64
Longitude                 float64
Cluster Labels              int64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

#### Map the clusters

In [152]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighbourhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow [cluster-1],
        fill=True,
        fill_color=rainbow [cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster exammination

#### Examine each cluster and determine the discriminating venue categories and name each cluster accordingly.


#### Cluster 1
#### The discrimminating venue categories are motels with multiethnic restuarants, probably its near a highway.
#### Cluster 1 - Near Highway enroute area

In [118]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"Cliffside, Cliffcrest, Scarborough Village West",0.0,Motel,American Restaurant,Vietnamese Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


#### Cluster 2
#### Discrimminating venues are southasian restuarants and general facilities.
#### Cluster 2 - South Asian community

In [119]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Guildwood, Morningside, West Hill",1.0,Rental Car Location,Electronics Store,Medical Center,Intersection,Bank,Restaurant,Mexican Restaurant,Breakfast Spot,College Stadium,Construction & Landscaping
4,Cedarbrae,1.0,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,College Stadium,Gym
5,Scarborough Village,1.0,Smoke Shop,Jewelry Store,Playground,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
7,"Golden Mile, Clairlea, Oakridge",1.0,Bakery,Bus Line,Ice Cream Shop,Park,Bus Station,Metro Station,Intersection,Soccer Field,Fried Chicken Joint,Coffee Shop
9,"Birch Cliff, Cliffside West",1.0,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Gym,Grocery Store,Gas Station,Fried Chicken Joint
10,"Dorset Park, Wexford Heights, Scarborough Town...",1.0,Indian Restaurant,Vietnamese Restaurant,Pet Store,Chinese Restaurant,Auto Garage,Bakery,American Restaurant,Grocery Store,General Entertainment,Gas Station
11,"Wexford, Maryvale",1.0,Accessories Store,Auto Garage,Bakery,Sandwich Place,Middle Eastern Restaurant,Smoke Shop,General Entertainment,Grocery Store,Clothing Store,Fried Chicken Joint
12,Agincourt,1.0,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Clothing Store,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Gas Station
13,"Clarks Corners, Tam O'Shanter, Sullivan",1.0,Pizza Place,Chinese Restaurant,Noodle House,Thai Restaurant,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Intersection,Bank,Italian Restaurant
14,"Milliken, Agincourt North, Steeles East, L'Amo...",1.0,Bakery,Playground,Park,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant


#### Cluster 3
####  Neighborhoods in cluster 3 discrimminate by having bars as the first most common venue and construction and landscaping offices.
#### Cluster 3 - Bar zone, Offices and shopping stores


In [120]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Rouge Hill, Port Union, Highland Creek",2.0,Bar,Construction & Landscaping,Vietnamese Restaurant,Clothing Store,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant


#### Cluster 4
#### Neighborhoods in cluster 4 have most restaurants from south east asia and general facilities. 
#### Cluster 4 - South East Asian Community

In [122]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Woburn,3.0,Coffee Shop,Korean Restaurant,Mexican Restaurant,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
6,"Kennedy Park, Ionview, East Birchmount Park",3.0,Train Station,Hobby Shop,Department Store,Coffee Shop,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint


#### Cluster 5
#### Area with most fast food restaurants and general facility.
#### Cluster 5 - General community

In [121]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Malvern, Rouge",4.0,Fast Food Restaurant,Vietnamese Restaurant,Clothing Store,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Department Store
